![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [241]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

In [242]:
# import os
# os.listdir('.')
# os.chdir('data')

In [243]:
df = pd.read_csv('car_reviews.csv',delimiter=';')

In [244]:
df.head()

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [245]:
df.shape

(5, 2)

In [246]:
# Start your code here!
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer
pipe = pipeline("sentiment-analysis",model=AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest'),tokenizer=AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest'))


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [247]:
pipe(df['Review'][0])

[{'label': 'positive', 'score': 0.9019393920898438}]

In [248]:
# Lets make the transformers pipeline use the default model for sentiment analysis
X = df['Review'].values
y = df['Class'].values

In [249]:
pipe = pipeline('sentiment-analysis')
predicted_labels = pipe(list(X))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [250]:
predictions = [pred['label']=='POSITIVE' for pred in predicted_labels]
labels = [pred=='POSITIVE' for pred in y]

In [251]:
import evaluate

accuracy_metric = evaluate.load('accuracy')
accuracy_result = accuracy_metric.compute(predictions=predictions,references=labels)['accuracy']

In [252]:
print(accuracy_result)

0.8


In [253]:
f1_metric=evaluate.load('f1')
f1_result = f1_metric.compute(predictions=predictions,references=labels)['f1']

In [254]:
print(f1_result)

0.8571428571428571


## Translation Task

In [255]:
from transformers import MarianMTModel

sentences = ['I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use.']
translatepipe = pipeline('translation',model=MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-es'),tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")) 

translated_review = translatepipe(sentences)[0]['translation_text']

Device set to use cpu


In [256]:
translated_review

'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'

In [257]:
references = open('reference_translations.txt').readlines()
print(references)

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.\n', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']


In [258]:

bleu_metric = evaluate.load('bleu')
bleu_score = bleu_metric.compute(predictions=[translated_review],references=[references])

In [259]:
print(bleu_score)

{'bleu': 0.7794483794144497, 'precisions': [0.9090909090909091, 0.8571428571428571, 0.75, 0.631578947368421], 'brevity_penalty': 1.0, 'length_ratio': 1.0476190476190477, 'translation_length': 22, 'reference_length': 21}


## QA Task

In [260]:
from transformers import AutoModelForQuestionAnswering

qapipe = pipeline('question-answering',model='deepset/minilm-uncased-squad2',tokenizer="deepset/minilm-uncased-squad2")


Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [261]:
context = "The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

question = 'What did he like about the brand?'

answer = qapipe(question=question,context=context)['answer']

In [262]:
print(answer)

ride quality, reliability


## Summarization task

In [263]:
X[-1]

"I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of lane changes. Th

In [264]:
summarypipe = pipeline('summarization')
summarized_text = summarypipe(X[-1],min_length=50,max_length=55)[0]['summary_text']

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [265]:
print(summarized_text)

 Nissan Rogue provides the desired SUV experience without burdening me with an exorbitant payment . Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more . The engine delivers strong performance, and
